In [1]:
failed_chat_ids = [
    "t316-943c8d19-2080-4bf8-bfd4-ffaec3e31b10",
    "t1256-a4029e29-8815-4df7-abd8-2a62947e5633",
    "t1276-22af6db8-3ced-44cf-a9b4-cd21146e05f1",
    "t313-ba62867d-4d37-47cf-9c07-13e4390e9586",
    "t1272-57c3b4ca-3d86-413f-9e1f-418e08076b45",
    "t1246-7c5fdac7-dab6-408d-9d57-56601c0e9523",
    "t1420-df50b175-e74d-4194-ae9c-ce517459a511",
    "t1262-c7c390cc-fb4d-4308-8c90-a354009303b2",
    "t1406-bc8fcdc0-0f8d-4f85-bcb0-6180ad32c990",
    "t1259-7b5b26fb-c0c1-4c1c-baab-09a19222b86e"
]

In [2]:
len(failed_chat_ids)

10

In [3]:
import requests
import json
import ast
def get_chat_log(x):
    url = "https://behnam-torob-ai-shopping-assistant.darkube.app/chat"
    messages = [{"type": "text", "content":  x}]
    data = {
            "chat_id": "check_chat_log",
            "messages": messages,
        }

        # Disable system proxy
    resp = requests.post(url, json=data, proxies={"http": None, "https": None})
    return resp

In [4]:
import ast
import datetime
from typing import Any
import re

import ast
import re

def clean_problematic_calls(s: str) -> str:
    """
    Replace problematic function calls (like datetime.datetime(...)) with safe strings
    before passing to literal_eval.
    """
    if not s:
        return s
    # Replace datetime.datetime(YYYY, MM, DD, hh, mm, ss) → "YYYY-MM-DD-hh-mm-ss"
    s = re.sub(
        r"datetime\.datetime\(([^)]+)\)",
        lambda m: "'" + m.group(1).replace(", ", "-") + "'",
        s
    )
    return s


def safe_str_to_list(s: str) -> list[str]:
    """
    Convert a string representation of a list into a list of strings safely.
    """
    if not s:
        return []
    s = clean_problematic_calls(s)
    try:
        result = ast.literal_eval(s)
        if isinstance(result, list):
            return [str(item) for item in result]
    except Exception as e:
        print("Failed to parse list string:", e)
    return []

def safe_str_to_dict(s: str) -> dict:
    """
    Convert a string representation of a Python dict into an actual dict,
    handling datetime and 'null' strings safely.
    """
    if not s:
        return {}
    s = s.replace("'null'", "None")
    s = re.sub(
        r"datetime\.datetime\(([^)]+)\)",
        lambda m: f"'{m.group(1).replace(', ', '-')}'",
        s
    )
    
    try:
        result = ast.literal_eval(s)
        if isinstance(result, dict):
            return result
    except Exception as e:
        print("Failed to parse string:", e)
    
    return {}

def parse_chat_log(raw: str) -> list[dict]:
    """
    Convert a raw string (list of dict-strings) into a list of dicts.
    """
    list_strs = safe_str_to_list(raw)
    return [safe_str_to_dict(item) for item in list_strs]

In [5]:
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
import base64
def visualize_iamge(image_b64):
    if image_b64.startswith("data:"):
        image_b64 = image_b64.split(",")[1]
    image_bytes = base64.b64decode(image_b64)
    image = Image.open(BytesIO(image_bytes))
    plt.imshow(image)
    plt.axis("off")
    plt.show()

In [6]:
def request_chat(x, image_url=None, chat_id = 'mytest'):

    url = "https://behnam-torob-ai-shopping-assistant.darkube.app/chat"
    messages = [{"type": "text", "content":  x}]
    if image_url:
        messages.append({"type": "image", "content": image_url})
    data = {
        "chat_id": chat_id,
        "messages": messages,
    }

    # Disable system proxy
    resp = requests.post(url, json=data, proxies={"http": None, "https": None})
    return resp.json()

In [7]:
from tqdm import tqdm
failed_dicts = []
for x in tqdm(sorted(failed_chat_ids, reverse=True)):
    chat_log = get_chat_log(x)
    chat_log_str = chat_log.json()['message']  # raw string
    chat_log_dicts = parse_chat_log(chat_log_str)
    failed_dicts.append(chat_log_dicts)

100%|██████████| 10/10 [00:09<00:00,  1.04it/s]


In [8]:
image_keys = ['user_image_url', 'model_image_url']
import random
import time

for i, chat_dict_list in enumerate(failed_dicts, 1):
    print(f"--- Chat #{i} ---")
    chat_id = f"chat_testv{int(random.random() * 10000)}"
    # Each chat is now a list of dicts (chat turns)
    for turn_idx, chat_dict in enumerate(chat_dict_list, 1):
        print(f" Turn {turn_idx}:")
        for key, value in chat_dict.items():
            if key not in image_keys:
                print(f"   {key}: {value}")
        
        # Show the image if available (byte content)
        text_input = chat_dict.get("user_text", "")
        if chat_dict.get("user_image_url"):
            try:
                img_bytes = chat_dict["user_image_url"]  # should be bytes
                # Example usage if you want to test sending it:

                visualize_iamge(img_bytes)  # <- fixed typo
            except Exception as e:
                print(f"   Failed to load image: {e}")
        else:
            # start = time.time()
            # resp = request_chat(text_input, chat_id = chat_id)
            # end = time.time()
            # elapsed = end - start
            # print(resp)
            # print(f"Elapsed time: {elapsed:.4f} seconds")
    print("\n")


IndentationError: expected an indented block after 'else' statement on line 25 (2952506293.py, line 32)